# Задания

### Модули

In [2]:
import matplotlib.pyplot as plt
import random
import sympy as smp
import numpy as np
import math
from ecpy.curves import Curve, Point

### Вспомогательные функции

In [6]:
def st(n):
    s = 0
    t = n
    while t % 2 == 0:
        s += 1
        t = t // 2
    return s, t

def Jacobi(a,n):
    if n < 0 or not n % 2:
        raise ValueError("n should be an odd positive integer") 
    j = 1
    if n == 1:
        return j
    if a<0:
        a = -a
        if n%4 == 3:
            j = -j
    while n>1:
        if a == 0:
            return 0
        s,t = st(a)
        if (s%2 == 1) & (n%8 in [3, 5]):
            j = -j
        if 3 == n%4 == t%4:
            j = -j
        a = n%t
        n = t
    return j

def Shanks(a, p):
    if not Jacobi(a, p) == 1:
        raise ValueError("a should be a quadratic residue")
    s, t = st(p-1)
    import random
    n = random.randint(2, p - 2)
    while Jacobi(n, p) == 1:
        n = random.randint(2, p - 2)
    b = pow(n, t, p)
    r = pow(int(a), (t+1)//2, p)
    d = 0
    f = pow(int(a), t, p)
    b2 = b
    for i in range(1,s):
        b2 = b2 * b2 % p
        if not pow(f, 2 ** (s - 1 - i), p) == 1:
            d += 2 ** i
            f = f * b2 % p 
    return r*pow(b, d//2, p) % p

### <font color='red'>Задание 1.</font>
Реализовать классы `ProjectiveCurve` и `ProjectivePoint`. Для объектов класса `ProjectivePoint` реализовать операции сложения и умножения на скаляр.

#### Решение

#### Классы

In [12]:
class ProjectiveCurve:
    """docstring"""
    
    def __init__(self, a, b, p):
        self.__a = a
        self.__b = b
        self.__p = p
        self.__pts = list(set(self.__get_points() + self.__get_shanks_points()))
        
    def get_coeffs(self):
        return self.__a, self.__b
    
    def get_mod(self):
        return self.__p
    
    def get_pts(self):
        return self.__pts
    
    def __get_points(self):
        X = [(x ** 3 + self.__a * x + self.__b) % self.__p for x in range(self.__p)]
        Y = [y ** 2 % self.__p for y in range(self.__p)]
        points = []
        for i in range(len(X)):
            for j in range(len(Y)):
                if(X[i] == Y[j]):
                    points += [[i, j]]
        points = [ProjectivePoint(self, pt[0], pt[1]) for pt in points]
        points += [ProjectivePoint(self, 0, 1, 0)]
        return points
    
    def __get_shanks_points(self):
        points = []
        for x in range(self.__p):
            a = (pow(x, 3, self.__p) + self.__a * x + self.__b) % self.__p
            if not Jacobi(a, self.__p) == 1:
                continue
            y = Shanks(a, self.__p)
            temp_point1 = ProjectivePoint(self, x, y)
            points += [temp_point1]
        return points
    
    def get_random_point(self):
        return self.__pts[random.randint(0, len(self.__pts) - 1)]
    
    def get_order(self):
        return len(self.__pts)
    
    def __repr__(self):
        return 'y^2 = x^3 + ({0}) * x + ({1}) mod {2}'.format(self.__a, self.__b, self.__p)

In [17]:
class ProjectivePoint():
    """docstring"""    
    
    def __init__(self, curve, x, y, z = 1):
        self.__x = x
        self.__y = y
        self.__z = z
        self.__curve = curve
        
    def get_x(self):
        return self.__x
    
    def get_y(self):
        return self.__y
    
    def get_z(self):
        return self.__z
    
    def get_curve(self):
        return curve
    
    def __repr__(self):
        return '({0}, {1}, {2})'.format(self.__x, self.__y, self.__z)
    
    def get_inverse_point(self):
        return ProjectivePoint(self.__curve, self.__x, -1 * self.__y)
    
    def __add__(self, other):
        a, b = self.__curve.get_coeffs()
        x1 = self.__x
        y1 = self.__y
        x2 = other.get_x()
        y2 = other.get_y()
        if (x1 == 0 and y1 == 0):
            return other
        elif(x2 == 0 and y2 == 0):
            return self
        if (x1 != x2):
            k = (y2 - y1) * pow((x2 - x1), -1, self.__curve.get_mod()) % self.__curve.get_mod()
            x3 = pow(k, 2, self.__curve.get_mod()) - x1 - x2
            y3 = k * (x1 - x3) % self.__curve.get_mod() - y1
        elif (x1 == x2 and y1 == y2 and y1 != 0):
            k = (3 * pow(x1, 2, self.__curve.get_mod()) % self.__curve.get_mod() + a) * pow(2 * y1, -1, self.__curve.get_mod()) % self.__curve.get_mod()
            x3 = pow(k, 2, self.__curve.get_mod()) - 2 * x1 % self.__curve.get_mod()
            y3 = k * (x1 - x3) % self.__curve.get_mod() - y1
        elif (x1 == x2 and y1 == y2 and y1 == 0):
            x3 = 0
            y3 = 0
        elif (x1 == x2 and y1 != y2):
            x3 = 0
            y3 = 0
        x3 %= self.__curve.get_mod()
        y3 %= self.__curve.get_mod()
        return ProjectivePoint(self.__curve, x3, y3)
        
    def __sub__(self, other):
        if isinstance(other, ProjectivePoint):
            return self + other.get_inverse_point()
        
    def __mul__(self, other):
        result = ProjectivePoint(self.__curve, 0, 0)
        Q = ProjectivePoint(self.__curve, self.__x, self.__y)
        m = bin(other)[2:]
        for i in reversed(range(len(m))):
            if (m[i] == '1'):
                result = result + Q
            Q = Q + Q
        return result
    
    def __eq__(self, other):
        return self.__x % self.__curve.get_mod() == other.get_x() % self.__curve.get_mod() and self.__y % self.__curve.get_mod() == other.get_y() % self.__curve.get_mod()
    
    def __hash__(self):
        return hash((self.__x, self.__y, self.__z))

#### Проверяю

In [18]:
curve1 = ProjectiveCurve(-1, 1, 13)

In [19]:
curve1.get_pts()

[(0, 1, 0),
 (7, 8, 1),
 (5, 11, 1),
 (5, 2, 1),
 (3, 8, 1),
 (4, 10, 1),
 (10, 9, 1),
 (7, 5, 1),
 (4, 3, 1),
 (3, 5, 1),
 (6, 9, 1),
 (12, 1, 1),
 (10, 4, 1),
 (12, 12, 1),
 (0, 1, 1),
 (0, 12, 1),
 (1, 12, 1),
 (6, 4, 1),
 (1, 1, 1)]

### <font color='red'>Задание 2.</font>
Проверить выполнение ассоциативности и коммутативности сложения точек эллиптической кривой в проективных координатах.

### <font color='red'>Задание 3.</font>
При помощи  кривой SECP256k1 сравнить скорость вычисления скалярного умножения в
- декартовых координатах;  
- проективных координатах.  

